# Part A - Van Gogh Painting Classifier

## Deep Learning Project - Tel Aviv University

This notebook implements a binary classifier to identify Van Gogh paintings using transfer learning with VGG19.

### Requirements:
- Google Colab with GPU runtime
- `post_impressionism_data.csv` file in the project (created by Get_Post_Impressionism_Data.ipynb)
- Weights & Biases account for experiment tracking

---


## 1. Environment Setup


In [ ]:
# Check environment
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Running on Google Colab")
else:
    print("Running locally")


✓ Running on Google Colab
✓ Connected via Git repository


In [ ]:
# Install required packages
%pip install -q optuna wandb


In [ ]:
# Dataset is already prepared - using post_impressionism_data.csv
# (Created by Get_Post_Impressionism_Data.ipynb)
print("✓ Using pre-prepared dataset from CSV file")


✓ Kaggle API configured successfully
  Username: avitalkrasovitskii
  Key: 53241c7887...


In [ ]:
# Dataset preparation skipped - using CSV file instead
# See Get_Post_Impressionism_Data.ipynb for dataset preparation
pass


Dataset URL: https://www.kaggle.com/datasets/steubk/wikiart
License(s): CC0-1.0


In [ ]:
# Dataset extraction skipped - using CSV file instead
pass


## 2. Import Libraries


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)

# Hyperparameter tuning and logging
import optuna
import wandb

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cpu':
    print("⚠️ WARNING: Running on CPU. For faster training, enable GPU in Colab:")
    print("   Runtime -> Change runtime type -> Hardware accelerator -> GPU")


In [ ]:
# Login to Weights & Biases
wandb.login(key="16d1bc863b28f81253ac0ee253b453393791a7e1")
print("✓ Logged in to Weights & Biases")


## 3. Data Preparation


In [ ]:
# Load dataset from CSV file
# The CSV file should be in the project directory or uploaded to Colab
import os
import pandas as pd

csv_paths = [
    "post_impressionism_data.csv",  # Current directory (if opened from GitHub)
    "/content/post_impressionism_data.csv",  # Colab
    "/kaggle/working/post_impressionism_data.csv",  # Kaggle
]

csv_path = None
for path in csv_paths:
    if os.path.exists(path):
        csv_path = path
        break

if csv_path is None:
    # Try to upload if in Colab
    if IN_COLAB:
        print("⚠️ CSV file not found in standard locations.")
        print("   Please upload post_impressionism_data.csv to Colab:")
        print("   Files -> Upload to session storage")
        try:
            from google.colab import files
            uploaded = files.upload()
            for filename in uploaded.keys():
                if filename.endswith('.csv'):
                    csv_path = filename
                    break
        except:
            pass
    
    if csv_path is None:
        raise FileNotFoundError(
            f"❌ CSV file not found!\n"
            f"  Please upload post_impressionism_data.csv or run Get_Post_Impressionism_Data.ipynb first.\n"
            f"  Checked: {csv_paths}"
        )

print(f"✓ Loading dataset from: {csv_path}")
df = pd.read_csv(csv_path)

print(f"\n✓ DataFrame loaded!")
print(f"  Total: {len(df)} images")
print(f"  Van Gogh: {df['is_van_gogh'].sum()}")
print(f"  Other: {len(df) - df['is_van_gogh'].sum()}")


In [ ]:
# OLD CODE - Replaced by Cell 12 (CSV loading)
# This cell kept for reference but not used
pass


In [ ]:
# Split dataset: 70% train, 15% validation, 15% test (stratified)

train_df, temp_df = train_test_split(
    df, test_size=0.3, stratify=df["is_van_gogh"], random_state=SEED
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df["is_van_gogh"], random_state=SEED
)

print("Dataset splits:")
print(f"  Train: {len(train_df):5d} images ({len(train_df)/len(df):.1%})")
print(f"  Val:   {len(val_df):5d} images ({len(val_df)/len(df):.1%})")
print(f"  Test:  {len(test_df):5d} images ({len(test_df)/len(df):.1%})")

print("\nClass distribution per split:")
print(f"  Train - Van Gogh: {train_df['is_van_gogh'].mean():.2%}")
print(f"  Val   - Van Gogh: {val_df['is_van_gogh'].mean():.2%}")
print(f"  Test  - Van Gogh: {test_df['is_van_gogh'].mean():.2%}")


## 4. Data Transforms & Dataset Class


In [ ]:
# Define transforms (ImageNet normalization for VGG19)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Training transforms with data augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Evaluation transforms (no augmentation)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print("Transforms defined ✓")


In [ ]:
# Custom Dataset class
class VanGoghDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["filepath"]).convert("RGB")
        label = row["is_van_gogh"]

        if self.transform:
            image = self.transform(image)

        return image, label

# Create datasets
train_dataset = VanGoghDataset(train_df, transform=train_transform)
val_dataset = VanGoghDataset(val_df, transform=eval_transform)
test_dataset = VanGoghDataset(test_df, transform=eval_transform)

print(f"Datasets created:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val:   {len(val_dataset)} samples")
print(f"  Test:  {len(test_dataset)} samples")


## 5. Training Functions


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    """Train model for one epoch. Returns average loss."""
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if wandb.run is not None:
            wandb.log({"batch_loss": loss.item()})

    return total_loss / len(loader)

@torch.no_grad()
def eval_one_epoch(model, loader, criterion, device):
    """Evaluate model. Returns average loss and accuracy."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(loader), correct / total

print("Training functions defined ✓")


## 6. Hyperparameter Tuning with Optuna


In [ ]:
def create_model(freeze_features=True):
    """Create VGG19 model with modified classifier for binary classification."""
    model = models.vgg19(weights='IMAGENET1K_V1')

    if freeze_features:
        for param in model.features.parameters():
            param.requires_grad = False

    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 2)
    return model.to(device)

def objective(trial):
    """Optuna objective function for hyperparameter optimization."""
    # Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    momentum = trial.suggest_float("momentum", 0.8, 0.99) if optimizer_name == "SGD" else 0.0
    freeze_features = trial.suggest_categorical("freeze_features", [True, False])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initialize W&B
    run = wandb.init(
        project="VanGogh_Classifier",
        name=f"trial_{trial.number}",
        config={"lr": lr, "optimizer": optimizer_name, "batch_size": batch_size,
                "weight_decay": weight_decay, "momentum": momentum,
                "freeze_features": freeze_features, "model": "VGG19"},
        reinit=True
    )

    # Create model and optimizer
    model = create_model(freeze_features=freeze_features)
    criterion = nn.CrossEntropyLoss()
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())

    if optimizer_name == "SGD":
        optimizer = torch.optim.SGD(trainable_params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = torch.optim.AdamW(trainable_params, lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.Adam(trainable_params, lr=lr, weight_decay=weight_decay)

    # Training loop
    num_epochs = 10
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)

        best_val_acc = max(best_val_acc, val_acc)

        wandb.log({"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss,
                   "val_acc": val_acc, "best_val_acc": best_val_acc})

        trial.report(val_acc, epoch)
        if trial.should_prune():
            run.finish()
            raise optuna.exceptions.TrialPruned()

    run.finish()
    return best_val_acc

print("Optuna objective function defined ✓")


In [ ]:
# Run hyperparameter search
print("Starting hyperparameter search...")
print("="*60)

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
)

# Adjust n_trials and timeout based on your time constraints
study.optimize(objective, n_trials=15, timeout=3600, show_progress_bar=True)

print("\n" + "="*60)
print("HYPERPARAMETER SEARCH COMPLETE!")
print("="*60)
print(f"\nBest validation accuracy: {study.best_value:.4f}")
print(f"\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")


## 7. Train Final Model


In [ ]:
# Get best parameters and train final model
best_params = study.best_params
print("Training final model with best parameters:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# Create DataLoaders
final_train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True, num_workers=2)
final_val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=2)

# Create model
final_model = create_model(freeze_features=best_params.get('freeze_features', False))

# Setup optimizer
criterion = nn.CrossEntropyLoss()
trainable_params = filter(lambda p: p.requires_grad, final_model.parameters())

if best_params['optimizer'] == "SGD":
    final_optimizer = torch.optim.SGD(trainable_params, lr=best_params['lr'],
                                       momentum=best_params.get('momentum', 0.9),
                                       weight_decay=best_params.get('weight_decay', 1e-4))
elif best_params['optimizer'] == "AdamW":
    final_optimizer = torch.optim.AdamW(trainable_params, lr=best_params['lr'],
                                         weight_decay=best_params.get('weight_decay', 1e-4))
else:
    final_optimizer = torch.optim.Adam(trainable_params, lr=best_params['lr'],
                                        weight_decay=best_params.get('weight_decay', 1e-4))

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(final_optimizer, mode='max', factor=0.5, patience=3, verbose=True)

# Initialize W&B
run = wandb.init(project="VanGogh_Classifier", name="final_model_training",
                 config={**best_params, "model": "VGG19", "training_type": "final"})

# Training loop with early stopping
num_epochs = 25
best_val_acc = 0.0
best_model_state = None
patience = 5
epochs_without_improvement = 0

train_losses, val_losses, val_accuracies = [], [], []

print("\nStarting training...")
print("="*60)

for epoch in range(num_epochs):
    train_loss = train_one_epoch(final_model, final_train_loader, final_optimizer, criterion, device)
    val_loss, val_acc = eval_one_epoch(final_model, final_val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    scheduler.step(val_acc)
    wandb.log({"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss,
               "val_acc": val_acc, "lr": final_optimizer.param_groups[0]['lr']})

    print(f"Epoch {epoch+1:02d}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = {k: v.cpu().clone() for k, v in final_model.state_dict().items()}
        epochs_without_improvement = 0
        print(f"  ✓ New best model! (Val Acc: {best_val_acc:.4f})")
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"\nEarly stopping after {epoch+1} epochs")
        break

final_model.load_state_dict(best_model_state)
print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.4f}")

torch.save({'model_state_dict': best_model_state, 'best_params': best_params,
            'best_val_acc': best_val_acc}, 'best_vangogh_classifier.pth')
print("Model saved to 'best_vangogh_classifier.pth'")

run.finish()


## 8. Visualizations


In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='Train Loss', color='#1f77b4', linewidth=2)
axes[0].plot(val_losses, label='Validation Loss', color='#ff7f0e', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(val_accuracies, label='Validation Accuracy', color='#2ca02c', linewidth=2, marker='o')
axes[1].axhline(y=best_val_acc, color='r', linestyle='--', label=f'Best: {best_val_acc:.4f}')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()
print("Saved: training_curves.png")


## 9. Test Set Evaluation


In [ ]:
# Evaluate on test set
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=2)

final_model.eval()
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        probs = torch.softmax(outputs, dim=1)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

all_preds, all_labels, all_probs = np.array(all_preds), np.array(all_labels), np.array(all_probs)

# Calculate metrics
test_accuracy = accuracy_score(all_labels, all_preds)
test_precision = precision_score(all_labels, all_preds)
test_recall = recall_score(all_labels, all_preds)
test_f1 = f1_score(all_labels, all_preds)
test_auc = roc_auc_score(all_labels, all_probs)

print("="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"\n📊 Metrics:")
print(f"   Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Precision: {test_precision:.4f}")
print(f"   Recall:    {test_recall:.4f}")
print(f"   F1-Score:  {test_f1:.4f}")
print(f"   AUC-ROC:   {test_auc:.4f}")

print(f"\n📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=['Not Van Gogh', 'Van Gogh']))


In [ ]:
# Confusion Matrix and ROC Curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Not Van Gogh', 'Van Gogh'],
            yticklabels=['Not Van Gogh', 'Van Gogh'], annot_kws={'size': 14})
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix', fontweight='bold')

fpr, tpr, _ = roc_curve(all_labels, all_probs)
axes[1].plot(fpr, tpr, color='#1f77b4', linewidth=2, label=f'ROC (AUC = {test_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='gray', linestyle='--')
axes[1].fill_between(fpr, tpr, alpha=0.2, color='#1f77b4')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('confusion_matrix_roc.png', dpi=150)
plt.show()
print("Saved: confusion_matrix_roc.png")


## 10. Final Summary


In [ ]:
# Final Summary
print("="*60)
print("PART A - VAN GOGH CLASSIFIER - SUMMARY")
print("="*60)

print("\n📁 Dataset:")
print(f"   Total: {len(df)} | Van Gogh: {df['is_van_gogh'].sum()} | Other: {len(df)-df['is_van_gogh'].sum()}")
print(f"   Split: 70% train / 15% val / 15% test")

print("\n🔧 Best Hyperparameters:")
for k, v in best_params.items():
    print(f"   {k}: {v}")

print("\n📊 Performance:")
print(f"   Best Val Acc:  {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Test F1-Score: {test_f1:.4f}")
print(f"   Test AUC-ROC:  {test_auc:.4f}")

print("\n💾 Saved Files:")
print("   best_vangogh_classifier.pth")
print("   training_curves.png")
print("   confusion_matrix_roc.png")

print("\n✅ Part A Complete!")
print("="*60)
